In [71]:
import pandas as pd
import numpy as np

# MERGING DATASETS

In [72]:
read_csv_parameters = {
    'parse_dates': ['date'],
    'dtype': {
        'county': 'string',
        'state': 'string',
        'fips': 'UInt64',
        'cases': 'UInt64',
        'deaths': 'UInt64'
    }
}

In [73]:
ppp_industry_merged=pd.read_csv('ppp_industry_merged.csv', index_col=0)
covid_data = pd.read_csv('ca_covid_data.csv', **read_csv_parameters)
tax_sales_data = pd.read_csv('county_loan_tax.csv',index_col=0)
zipcode_data = pd.read_csv('ca_zipcodes.csv',index_col=0)
employment_data = pd.read_csv('ca_counties_employment_2020.csv',index_col=0)

In [74]:
ppp_industry_merged.head()

,loan_amount,business_name,address,city,state,zipcode,naics_code,business_type,jobs_reported,date_approved,lender,cd,industry_title,two_digit_code,general_industry_title,thousands_of_jobs_2019
0,10000000,ito incorporated,431 isis ave,inglewood,ca,90301,722310,subchapter s corporation,500,2020-04-09,city national bank,ca-43,food service contractors,72,leisure and hospitality,16575.9
1,6634900,"oh my green, inc.",1845 rollins rd,burlingame,ca,94010,722310,corporation,200,2020-04-14,silicon valley bank,ca-14,food service contractors,72,leisure and hospitality,16575.9
2,6052100,eat club inc.,1400 seaport blvd,redwood city,ca,94063,722310,corporation,500,2020-04-30,comerica bank,ca-14,food service contractors,72,leisure and hospitality,16575.9
3,5376800,"bld brands, llc",20377 sw acacia st. suite 200,newport beach,ca,92660,722310,limited liability company(llc),500,2020-04-08,"jpmorgan chase bank, national association",ca-48,food service contractors,72,leisure and hospitality,16575.9
4,4764922,"z golf food & beverage services, llc",43385 business park drive,temecula,ca,92590,722310,limited liability company(llc),500,2020-04-05,"wintrust bank, national association",ca-42,food service contractors,72,leisure and hospitality,16575.9


In [75]:
covid_data=covid_data[covid_data['county'].notna()]

covid_data

,date,county,state,fips,cases,deaths
0,2020-01-25,orange,ca,6059,1,0
1,2020-01-26,los angeles,ca,6037,1,0
2,2020-01-26,orange,ca,6059,1,0
3,2020-01-27,los angeles,ca,6037,1,0
4,2020-01-27,orange,ca,6059,1,0
...,...,...,...,...,...,...
14476,2020-11-21,tulare,ca,6107,19673,303
14477,2020-11-21,tuolumne,ca,6109,708,8
14478,2020-11-21,ventura,ca,6111,17741,174
14479,2020-11-21,yolo,ca,6113,4139,75


In [76]:
covid_data=covid_data[covid_data['county'].notna()]

In [77]:
apr_10_covid_data=covid_data[covid_data['date']==pd._libs.tslibs.timestamps.Timestamp('2020-04-10 00:00:00')]

In [78]:
list(set(covid_data['county']).difference(set(apr_10_covid_data['county'])))

['mariposa', 'sierra', 'lassen', 'trinity', 'modoc']

In [79]:
no_case_counties=[('mariposa', 6043), ('sierra', 6091), ('trinity', 6105), ('modoc', 6049), ('lassen', 6035)]

for (county, fips) in no_case_counties:
    apr_10_covid_data.loc[len(apr_10_covid_data)] = [pd._libs.tslibs.timestamps.Timestamp('2020-04-10 00:00:00'), county, 'ca', fips, 0,0]

apr_10_covid_data=apr_10_covid_data.reset_index(drop=True)

len(apr_10_covid_data)


/Users/ayeshayusuf/.local/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


58

In [80]:
tax_sales_data.head()

,county,loan_amount,q1,q2,q_dif
0,alameda,3737714415,20130869785,17192629309,-2938240476
1,alpine,582354,18256845,9786734,-8470111
2,amador,26011385,305561596,341544440,35982844
3,butte,267245789,2329470124,2500864025,171393901
4,calaveras,33918628,269695690,330519746,60824056


In [81]:
zipcode_data.head()

,zipcode_type,city,county
zipcode,,,
90001,non-unique,los angeles,los angeles
90002,non-unique,los angeles,los angeles
90003,non-unique,los angeles,los angeles
90004,non-unique,los angeles,los angeles
90005,non-unique,los angeles,los angeles


In [82]:
employment_data.head()

,area_name,date,year,month,labor_force,employment,unemployment,unemployment_rate
0,alameda county,2020-01-01,2020,january,841300,815500,25700,0.031
1,alpine county,2020-01-01,2020,january,650,620,30,0.046
2,amador county,2020-01-01,2020,january,14860,14160,700,0.047
3,butte county,2020-01-01,2020,january,97800,92400,5400,0.056
4,calaveras county,2020-01-01,2020,january,21560,20630,930,0.043


In [83]:
employment_data = employment_data.rename(columns={'area_name':'county'})
employment_data['county'] = employment_data['county'].str.rstrip('county')

In [84]:
zipcode_merged = ppp_industry_merged.merge(zipcode_data, on='zipcode')

tax_sales_merged = zipcode_merged.merge(tax_sales_data, on='county')

final_merged = tax_sales_merged.merge(apr_10_covid_data, on='county')

In [85]:
final_merged=final_merged.drop(['city_y', 'state_y'], axis=1).rename(columns={'loan_amount_x':'ppp_loan_amount','city_x':'city', 'state_x':'state', 'loan_amount_y':'county_total_loan_amount', 'cases':'cases_apr10', 'deaths':'deaths_apr10'})
final_merged

,ppp_loan_amount,business_name,address,city,state,zipcode,naics_code,business_type,jobs_reported,date_approved,...,zipcode_type,county,county_total_loan_amount,q1,q2,q_dif,date,fips,cases_apr10,deaths_apr10
0,10000000,ito incorporated,431 isis ave,inglewood,ca,90301,722310,subchapter s corporation,500,2020-04-09,...,non-unique,los angeles,18670084713,103484958447,88778057202,-14706901245,2020-04-10,6037,8430,241
1,122194,sweet_&_chilli_llc,440 hindry ave unit h,inglewood,ca,90301,722511,limited liability company(llc),7,2020-06-16,...,non-unique,los angeles,18670084713,103484958447,88778057202,-14706901245,2020-04-10,6037,8430,241
2,101041,decadent cuising kitchen,936 w manchester blvd unit b,inglewood,ca,90301,722511,sole proprietorship,6,2020-08-06,...,non-unique,los angeles,18670084713,103484958447,88778057202,-14706901245,2020-04-10,6037,8430,241
3,87210,el puerto escondido inc,915 arbor vitae,inglewood,ca,90301,722511,corporation,0,2020-05-02,...,non-unique,los angeles,18670084713,103484958447,88778057202,-14706901245,2020-04-10,6037,8430,241
4,66400,441 east corporation,441 e manchester blvd,inglewood,ca,90301,722511,corporation,12,2020-04-15,...,non-unique,los angeles,18670084713,103484958447,88778057202,-14706901245,2020-04-10,6037,8430,241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456784,10000,stonefly restaurant,14821 highway 89,markleeville,ca,96120,722511,sole proprietorship,6,2020-05-04,...,non-unique,alpine,582354,18256845,9786734,-8470111,2020-04-10,6003,2,0
456785,21455,"mad dog cafe, llc",290 old pony express rd a,markleeville,ca,96120,722515,limited liability company(llc),8,2020-04-11,...,non-unique,alpine,582354,18256845,9786734,-8470111,2020-04-10,6003,2,0
456786,6776,day michael k construction inc,22001 state route 88,markleeville,ca,96120,236115,corporation,0,2020-06-01,...,non-unique,alpine,582354,18256845,9786734,-8470111,2020-04-10,6003,2,0
456787,3041,"denise gemmill, tc",19 aspen way,markleeville,ca,96120,812990,independent contractors,1,2020-07-16,...,non-unique,alpine,582354,18256845,9786734,-8470111,2020-04-10,6003,2,0


In [86]:
final_merged.to_csv('final_combined_df.csv')